In [1]:
%load_ext autoreload
%autoreload 2
%run notebook_setup.py

Working directory set to: C:\pbgui\passivbot
Python path: ['C:\\pbgui\\passivbot\\src', 'C:\\pbgui\\passivbot', 'C:\\pbgui\\passivbot\\notebooks', 'C:\\Python39\\python39.zip', 'C:\\Python39\\DLLs', 'C:\\Python39\\lib', 'C:\\Python39', 'C:\\pbgui\\venv', '', 'C:\\pbgui\\venv\\lib\\site-packages', 'C:\\pbgui\\venv\\lib\\site-packages\\win32', 'C:\\pbgui\\venv\\lib\\site-packages\\win32\\lib', 'C:\\pbgui\\venv\\lib\\site-packages\\Pythonwin']


In [2]:
import sys
import os
from backtest_forager import *

using numba


In [3]:
config = load_config(r"C:\pbgui\passivbot\opt_results_forager_analysis\2024-08-10T08_36_44_39_coins_f0378ac1.json")
{k: config[k] for k in ['backtest', 'bot', 'live']}

Exception: failed to load config C:\pbgui\passivbot\opt_results_forager\2024-08-09T17_36_15_39_coins_1c1a424e_all_results.txt: failed to load config file C:\pbgui\passivbot\opt_results_forager\2024-08-09T17_36_15_39_coins_1c1a424e_all_results.txt Extra data: line 2 column 1 - line 9499 column 1 (char 5824 - 55292483)

In [ ]:
symbols = config['backtest']['symbols']
hlcs, mss, results_path = await prepare_hlcs_mss(config)
preferred_coins = calc_preferred_coins(hlcs, config)

In [ ]:
fills, equities, analysis = run_backtest(hlcs, preferred_coins, mss, config)

In [ ]:
sts = utc_ms()
fdf = process_forager_fills(fills)
print(f'elapsed {utc_ms() - sts}')
sts = utc_ms()
equities = pd.Series(equities)
analysis_py, balance_and_equity = analyze_fills_forager(config['backtest']['symbols'], hlcs, fdf, equities)
for k in analysis_py:
    if k not in analysis:
        analysis[k] = analysis_py[k]
print(f'elapsed {utc_ms() - sts}')
pprint.pprint(analysis)
from datetime import datetime, timedelta
base_date = datetime.strptime(config['backtest']['start_date'], '%Y-%m-%d') 
balance_and_equity.index = pd.to_timedelta(balance_and_equity.index, unit='m') + base_date
balance_and_equity.plot()

In [ ]:
syms_sorted_by_volume = fdf.groupby('symbol').fee_paid.sum().sort_values().index.to_list()
for i, symbol in enumerate(syms_sorted_by_volume[:5]):
    print(f"Plotting fills for {symbol}")
    hlcs_df = pd.DataFrame(hlcs[:, symbols.index(symbol), :], columns=["high", "low", "close"])
    fdfc = fdf[fdf.symbol == symbol]
    plt.clf()
    plot_fills_forager(fdfc, hlcs_df)
    plt.title(f"Fills {symbol}")
    plt.xlabel = "time"
    plt.ylabel = "price"
    plt.show()
